<h1>Эксперименты с эмбеддингами</h1>

Для начала приведу текст к чистому состоянию.

In [3]:
data = ' '.join(open('Пугачев.txt').readlines())

alphabet = 'абвгдеёжзийклмнопртсухфцчшщьыьэюяabcdefghijklmnopqrstuvwxyz'
stop_signs = '.!?'

def looks_like_word(text):
  ok = False
  for letter in alphabet:
    if letter in text.lower():
      ok = True
      break
  return ok

def stop_separator(text):
  sep = ''
  for stop in stop_signs:
    if stop in text.lower():
      sep = stop
      break
  return sep

sentences = ['']
for token in data.split():
  if looks_like_word(token):
    sign_inside = stop_separator(token)
    if len(sign_inside) == 0:
      sentences[-1] += token + ' '
    else:
      mas = token.split(sign_inside)
      for elem in range(len(mas)):
        sentences[-1] += mas[elem] + ' '
        if elem + 1 < len(mas):
          sentences.append('')
  else:
    if len(stop_separator(token)) != 0:
      sentences.append('')


clear_sentences = []

for sentence in sentences:
  clear_sentences.append('')
  for symbol in sentence:
    if symbol.isalpha() or symbol == ' ':
      clear_sentences[-1] += symbol
  clear_sentences[-1] = clear_sentences[-1].split()

clear_sentences[20:30:5]

[['Побег', 'кочующего', 'народа'],
 ['Его',
  'течение',
  'быстро',
  'мутные',
  'воды',
  'наполнены',
  'рыбою',
  'всякого',
  'рода',
  'берега',
  'большею',
  'частию',
  'глинистые',
  'песчаные',
  'и',
  'безлесные',
  'но',
  'в',
  'местах',
  'поемных',
  'удобные',
  'для',
  'скотоводства']]

По имеющимся данным попытаюсь построить эмбеддинги (чем ближе слова будут в координатном пространстве, тем более подхоядщими по смыслу они будут, тем чаще идут вместе).

In [3]:
import gensim.downloader as api
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec

w2v_model = Word2Vec(clear_sentences, min_count=0, workers=cpu_count())

print(w2v_model.wv['любовь'])

[-1.8512619e-03 -4.3107467e-03 -5.0928579e-03 -7.8272948e-04
 -3.3063896e-04 -4.5096874e-03 -2.9040745e-03 -2.0636597e-03
 -2.6016887e-03 -3.5684090e-03 -1.6257756e-03  6.2840199e-03
 -1.8738741e-03  3.1306907e-03 -6.2011200e-04 -1.6537684e-03
 -5.5663255e-03  4.0362594e-03  2.7452693e-03  3.6764906e-03
  2.6353071e-03 -6.6098175e-04  4.2350036e-03 -3.6084035e-05
  8.6874969e-04  3.3424343e-03  5.4005906e-03  6.7211138e-03
 -4.2289188e-03 -1.5309694e-03  4.9110330e-03  1.9595632e-03
  3.4423696e-03  2.7240783e-03  7.1146311e-03  1.7185184e-03
 -1.4193208e-03 -1.6993393e-04  3.1732544e-03  2.8022041e-03
  1.1188762e-03  2.3828281e-04  2.1661888e-03 -1.3110507e-03
  3.7518109e-03 -8.8370702e-04  4.1199005e-03 -5.1272907e-03
  3.0157086e-03  2.5700536e-03 -4.5978134e-03  6.9370517e-04
  1.6258479e-04 -1.2268510e-03  2.8098153e-03 -4.9290241e-04
 -2.5097610e-04 -7.4694760e-04  2.9140597e-03  5.2233478e-03
 -6.6767554e-03  2.8783802e-04 -6.0377113e-04 -5.9251124e-03
  2.7222012e-04 -9.30982

In [4]:
print(w2v_model.wv.most_similar('Бунт'))

[('петлю', 0.3383217453956604), ('казней', 0.333809494972229), ('подымаются', 0.33150309324264526), ('нашествием', 0.3243948519229889), ('баталиона', 0.3240799903869629), ('Надеемся', 0.3216485381126404), ('Царицын', 0.3101961016654968), ('приказу', 0.30966800451278687), ('Действия', 0.30502933263778687), ('смену', 0.3028341233730316)]


In [5]:
print(w2v_model.wv.most_similar('татары'))

[('нашему', 0.7033239006996155), ('спешить', 0.7007172107696533), ('самозванцу', 0.6941964626312256), ('силами', 0.6919719576835632), ('отрядом', 0.6898075938224792), ('станицу', 0.6849314570426941), ('деревни', 0.682374119758606), ('октября', 0.6753102540969849), ('пушек', 0.6743494868278503), ('губернатору', 0.6735434532165527)]


Получил набор синонимичных слов, они действительно могут идти где-то рядом в тексте или служить заменой исследуемому слову, но неясно, как это использовать в автодополнении запроса.

Попробую похожий подход, но в векторное пространство переведу не отдельные слова, а целые предлжения.

In [6]:
import gensim
from gensim.models import doc2vec

def tagged_document(list_of_ListOfWords):
    for x, ListOfWords in enumerate(list_of_ListOfWords):
        yield doc2vec.TaggedDocument(ListOfWords, [x])

data_train = list(tagged_document(clear_sentences))

d2v_model = doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)

d2v_model.build_vocab(data_train)

d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

vec1 = d2v_model.infer_vector(['я', 'вас', 'любил'])
print(vec1)

[-0.0615301  -0.11173794 -0.02546872  0.06077589 -0.07444113 -0.00151369
  0.01513805  0.08924188  0.00573151 -0.12549391 -0.06643118  0.01989817
 -0.12096556  0.0127247  -0.02978231 -0.05801844  0.04731642 -0.03068523
  0.09391801 -0.05286277 -0.1015524  -0.0136481  -0.01362483 -0.02586907
  0.16192424 -0.03509224 -0.01037985 -0.02852956  0.03948101  0.04571668
  0.10232787  0.05417688  0.03177855  0.03089807  0.03671477 -0.02773746
  0.12772171 -0.03518483  0.00227068  0.1880481 ]


In [13]:
embedded_sentences = {}
for sentence in clear_sentences:
  embedded_sentences[' '.join(sentence)] = d2v_model.infer_vector(sentence)
embedded_sentences['Побег кочующего народа']

array([-0.05150026, -0.1248804 , -0.03152464,  0.08531298, -0.07933158,
       -0.01972366,  0.03062134,  0.08424878,  0.03676644, -0.14537479,
       -0.06926925,  0.03498233, -0.1225356 ,  0.00997425, -0.03887725,
       -0.09231061,  0.03826494, -0.00603526,  0.12281232, -0.04735072,
       -0.09486103, -0.02162586, -0.0151908 , -0.01326367,  0.16914855,
       -0.0301776 , -0.00780203, -0.03970532,  0.04878021,  0.06405453,
        0.07803874,  0.04559334,  0.02327166,  0.06015848,  0.04130342,
       -0.01448507,  0.13810274, -0.04107934,  0.01790597,  0.18583952],
      dtype=float32)

In [7]:
import numpy as np

vec2 = d2v_model.infer_vector(['я', 'вас', 'любил'])
print(np.linalg.norm(vec1 - vec2))

0.0623445


In [8]:
vec3 = d2v_model.infer_vector(['ночь', 'улица', 'фонарь', 'аптека'])
print(np.linalg.norm(vec1 - vec3))

0.12904842


Чем более различны текста - тем больше норма расстояния между ними.

Чтобы использовать полученные данные в автодополнении, для введённой фразы буду искать наиболее близкую к ней из исходгого набора текста.

In [28]:
def get_k_nearest_neighbors(sentence, k):
  sentence = d2v_model.infer_vector(sentence.lower().split())
  return list(zip(*sorted(list(map(lambda key: (np.linalg.norm(sentence - embedded_sentences[key]), key), embedded_sentences.keys())))))[1][:k]

In [29]:
get_k_nearest_neighbors('Сгоревшие деревянные укрепления были заменены снеговыми', 5)

('Несколько четвертей муки и сухарей валялись на дворе',
 'Перфильев перекрестясь простерся ниц и остался недвижим',
 'Одни казачки были приняты',
 'Деревни башкирские были пусты',
 'Распоряжение Рейнсдорпа')

Построение дало довольно странный результат. Видимо, влияет недостаток входных данных. Одако же для более крупных текстов данный подход слабо применим, так как в момент, когда пользователь делает запрос, нужно перебрать расстояния до всех имеющихся предложений. Это довольно долго.